In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [3]:
lv = LV.from_pyvista_read("../sample_files/lvtetmesh_coarse.vtk")
# lv.smooth_surface(n_iter=500)
lv.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)
lv.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value,
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [4]:
lv.compute_fibers(surfRegionsIds=LV_FIBER_MODES.LDRB_1,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={},
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

ImportError: ldrb library is required for fiber computation.                Please, see https://github.com/finsberg/ldrb for details.

In [ ]:
lv.mesh.save("../sample_files/lvtetmesh_fibers_LDRB_1.vtk")

In [ ]:
from project_heart.utils.vector_utils import *
fibers = lv.mesh.point_data[LV_FIBERS.F0.value]
normal_vec = np.repeat(np.expand_dims(lv.get_normal(),1),len(fibers), axis=1).T
angles = angle_between(fibers, normal_vec, check_orientation=False) - np.pi*0.5
angles = np.degrees(angles)
lv.mesh.point_data["FIBER_ANGLES"] = angles

In [ ]:
seed_mesh = lv.mesh.decimate_boundary(0.95)
seed_mesh

In [ ]:
stream = lv.mesh.streamlines_from_source(seed_mesh, 
                                         vectors=LV_FIBERS.F0.value,
                                        #  surface_streamlines=True,
                                         max_steps=2000,
                                         terminal_speed=0.0,
                                         integration_direction="both",
                                         )
stream

In [ ]:
p = pv.Plotter()
p.enable_anti_aliasing()
p.enable_shadows()
p.background_color = 'w'
# p.add_mesh(lv.mesh, color="beige")
p.add_mesh(stream, scalars="FIBER_ANGLES", lighting=False)
p.show()